In [ ]:
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [ ]:
pd.set_option('display.max_rows',50)
pd.set_option('display.max_colwidth', 20)

Load and assign data types

In [ ]:

with open('../data/data_final_dtypes.json', 'r') as f:
    dtypes = json.load(f)

In [ ]:
df = pd.read_csv('../data/data_final.csv', dtype=dtypes)

In [ ]:
df.date = pd.to_datetime(df.date)

Import fips codes for states and counties

In [ ]:
geocodes = pd.read_excel('../data/geodata/all-geocodes-v2018.xlsx', header=4, dtype=str)

In [ ]:
geocodes.head(2)

In [ ]:
geocodes.rename({'State Code (FIPS)': 'state_fips', 
                 'County Code (FIPS)': 'county_fips',
                 'County Subdivision Code (FIPS)': 'sub_county_fips',
                 'Place Code (FIPS)': 'place_fips',
                 'Area Name (including legal/statistical area description)': 'area'},
               axis=1, inplace=True)

In [ ]:
geocodes = geocodes[['state_fips', 'county_fips', 'sub_county_fips', 'place_fips','area']].iloc[1:, :]

In [ ]:
geocodes.head(2)

In [ ]:
states = geocodes.drop_duplicates(subset='state_fips').reset_index()

In [ ]:
county_fips_long = geocodes.state_fips + geocodes.county_fips

In [ ]:
geocodes.insert(2, 'county_fips_long', county_fips_long)

In [ ]:
counties = geocodes.query('county_fips != "000"').drop_duplicates(subset='county_fips_long')

In [ ]:
counties = counties[counties.state_fips != '72'] # drop Puerto Rico

In [ ]:
counties['area_key'] = counties.area.str.rsplit(' ', 1).str[0].str.lower()

In [ ]:
# make column with state codes
state_fips = pd.read_csv('../data/geodata/states_fips.csv', dtype=str)

In [ ]:
state_fips.head()

In [ ]:
state_fips = state_fips[['st', 'stusps']]

In [ ]:
state_fips.stusps = state_fips.stusps.str.strip()
state_fips.st = state_fips.st.str.strip()

In [ ]:
state_fips = state_fips.set_index('st').to_dict()['stusps']

In [ ]:
counties['state_code'] = counties.state_fips.map(state_fips)

In [ ]:
counties

In [ ]:

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties_json = json.load(response)

dfips = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

In [ ]:
# fig = px.choropleth(geojson=counties_json, locations=counties.county_fips_long,
#                            color_continuous_scale="Viridis",
#                            range_color=(0, 12),
#                            scope="usa",
#                            labels={'unemp':'unemployment rate'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

Add fips code information for all the counties and states in the main dataset

In [ ]:
df['county_key'] = df.county.str.lower() 

In [ ]:
df_fips = pd.merge(df, counties, how='left', 
                   left_on=['county_key', 'state'], right_on=['area_key', 'state_code'])

In [ ]:
df_fips = df_fips.drop_duplicates(subset=['name', 'date', 'county', 'state'])

In [ ]:
df_fips.head()

## Killings per County 2000-2021

Get total killings per county - all-time

In [ ]:
kills_per_county = (df_fips.groupby(['county_fips_long', 'county', 'state'])['name']
         .count()
         .reset_index()
         .rename({'name': 'kills_total'}, axis=1)
        )

In [ ]:
# kills_per_county.head()

In [ ]:
# fig = px.choropleth(kills_per_county, geojson=counties_json, locations='county_fips_long',
#                     color='kills_total',
#                     hover_data=['county', 'kills_total'],
#                            color_continuous_scale="Viridis",
#                            scope="usa",
#                            labels={'unemp':'unemployment rate'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

Merge `kills_per_county` dataframe with population per county info

In [ ]:
pop = pd.read_csv('../data/geodata/co-population-est2019-alldata.csv',
                  encoding = "ISO-8859-1", 
                  dtype=str, 
                  usecols=['STATE', 'STNAME', 'CTYNAME', 'COUNTY', 
                           'CENSUS2010POP', 'ESTIMATESBASE2010',
                          'POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 
                          'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015', 
                          'POPESTIMATE2016', 'POPESTIMATE2017', 'POPESTIMATE2018',
                          'POPESTIMATE2019'])

In [ ]:
# Average the population per county for 2010 - 2019 estimates
pop_estimates = pop.iloc[:, 4:].apply(pd.to_numeric, axis=1)

In [ ]:
pop_avrg = pop.iloc[:, :4]

In [ ]:
pop_avrg.insert(4, 'pop_avrg', pop_estimates.mean(axis=1))

In [ ]:
pop_avrg.insert(2, 'county_fips', pop_avrg.STATE + pop_avrg.COUNTY)

In [ ]:
pop_avrg_min = pop_avrg[['county_fips', 'pop_avrg', 'STNAME']]

In [ ]:
pop_avrg_min.head()

In [ ]:
# Now we are ready to merge with killings
kills_county_pop = kills_per_county.merge(pop_avrg_min, how='left', 
                                         left_on='county_fips_long',
                                         right_on='county_fips', indicator=True)

In [ ]:
kills_county_pop.pop_avrg = kills_county_pop.pop_avrg.astype(int)

In [ ]:
kills_county_pop.rename({'county_fips_long': 'fips', 'STNAME': 'state_name'}, 
                       axis=1, inplace=True)

In [ ]:
kills_county_pop.drop(labels=['county_fips', '_merge'], axis=1, inplace=True)

In [ ]:
kills_per_1000 = kills_county_pop.kills_total / kills_county_pop.pop_avrg * 1000 

In [ ]:
kills_county_pop.insert(3, 'kills_per_1000', kills_per_1000)

In [ ]:
fig = px.choropleth(kills_county_pop, geojson=counties_json, locations='fips',
                    color='kills_per_1000',
                    hover_data=['county', 'pop_avrg', 'kills_total', 'kills_per_1000'],
                           color_continuous_scale="Viridis",
                           scope="usa",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


## Study Crime Data

In [ ]:
# https://www.kaggle.com/mikejohnsonjr/united-states-crime-rates-by-county
# Crime (2016): https://www.icpsr.umich.edu/icpsrweb/
# Population (2013): https://census.gov
crime = pd.read_csv('../data/geodata/crime_data_w_population_and_crime_rate_per_county.csv', dtype=str)

In [ ]:
crime = crime[['county_name', 'population','crime_rate_per_100000', 'MURDER', 'RAPE', 
              'AGASSLT', 'BURGLRY', 'FIPS_ST', 'FIPS_CTY']]

In [ ]:
crime.columns

In [ ]:
# correct the fips codes so they all have the same number of digits. i.e add zeros to the front of the string
def correct_fips(s, length=3):
    while len(s) < length:
        s = '0' + s
    return s

crime.FIPS_ST = crime.FIPS_ST.apply(correct_fips, args=[2])
crime.FIPS_CTY = crime.FIPS_CTY.apply(correct_fips, args=[3])

In [ ]:
crime['fips_county'] = crime.FIPS_ST + crime.FIPS_CTY

In [ ]:
crime.iloc[:, 1:7] = crime.iloc[:, 1:-1].apply(pd.to_numeric)

In [ ]:
crime.crime_rate_per_100000 = crime.crime_rate_per_100000.astype(int)

In [ ]:
violent = crime.loc[:, 'MURDER':'BURGLRY'].sum(axis=1)/crime.population * 100000

In [ ]:
crime.insert(3, 'violent_crimes_per_100000', violent.astype(int))

In [ ]:
crime

In [ ]:
# fig = px.choropleth(crime, geojson=counties_json, locations='fips_county',
#                     color='violent_crimes_per_100000',
#                     hover_data=['county_name', 'population', 'violent_crimes_per_100000'],
#                            color_continuous_scale="Viridis",
#                            scope="usa",
#                            labels={'unemp':'unemployment rate'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [ ]:
# make quantiles for the crime-rates
crime.violent_crimes_per_100000.quantile([.2, .4, .6, .8, 1])

In [ ]:
crime.violent_crimes_per_100000.describe(percentiles=[.2, .4, .6, .8, 1])

In [ ]:
cq = pd.qcut(crime.violent_crimes_per_100000, q=4)
crime['vc_per100k_quant'] = cq

In [ ]:
colormap = ['#edf8fb','#bfd3e6','#9ebcda','#8c96c6','#8856a7','#810f7c']
fig = px.choropleth(crime, geojson=counties_json, locations='fips_county',
                    color='vc_per100k_quant',
                    hover_data=['county_name', 'population', 'violent_crimes_per_100000'],
                           color_discrete_sequence=colormap[::-1],
                           scope="usa",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
cq

In [ ]:
kills_q = pd.qcut(kills_county_pop.kills_per_1000, q=4)
kills_county_pop['kills_quant'] = kills_q

In [ ]:
colormap = ['#edf8fb','#bfd3e6','#9ebcda','#8c96c6','#8856a7','#810f7c']
fig = px.choropleth(kills_county_pop.sort_values(by='kills_quant', ascending=False), 
                    geojson=counties_json, locations='fips',
                    color='kills_quant',
                    hover_data=['county', 'pop_avrg', 'kills_total', 'kills_per_1000'],
                           color_discrete_sequence=colormap[::-1],
                           scope="usa",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


In [ ]:
# Merge crime rates with killings
dm = kills_county_pop.merge(crime, how='left', left_on='fips', right_on='fips_county')

In [ ]:
dm.head(1)

In [ ]:
cq = pd.qcut(dm.violent_crimes_per_100000, q=4)
dm['vc_per100k_quant'] = cq

In [ ]:
dm.dropna(subset=['fips_county'], inplace=True)

In [ ]:
colormap = ['#edf8fb','#bfd3e6','#9ebcda','#8c96c6','#8856a7','#810f7c']
fig = px.choropleth(dm.sort_values(by='vc_per100k_quant', ascending=False), 
                    geojson=counties_json, locations='fips',
                    color='vc_per100k_quant',
                    hover_data=['county', 'pop_avrg', 'kills_per_1000'],
                           color_discrete_sequence=colormap[::-1],
                           scope="usa",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
df_fips.head(2)

## Killings per State 2000-2021

In [ ]:
kills_per_state = (df_fips.groupby(['state_fips', 'state'])['name']
         .count()
         .reset_index()
         .rename({'name': 'kills_total'}, axis=1)
        )

In [ ]:
kills_per_state.head()

In [ ]:
pop_avrg.head(2)

In [ ]:
kills_state_pop = kills_per_state.merge(pop_avrg, how='left', 
                                     left_on='state_fips',
                                     right_on='STATE', indicator=True)

In [ ]:
kills_state_pop = kills_state_pop.drop_duplicates(subset=['state', 'state_fips']).reset_index().drop('index', axis=1)

In [ ]:
kills_state_pop = kills_state_pop[['state_fips', 'state', 'kills_total', 'STNAME', 'pop_avrg']]

In [ ]:
kills_state_pop.head(2)

In [ ]:
# merge crimes dataset too
crime_state = crime.groupby('FIPS_ST')[['population', 'crime_rate_per_100000', 
                          'MURDER', 'RAPE', 'AGASSLT', 'BURGLRY']].sum().reset_index()

In [ ]:
kills_state_crime = kills_state_pop.merge(crime_state, how='left', left_on='state_fips', right_on='FIPS_ST')

In [ ]:
violent_crimes = kills_state_crime.loc[:, 'MURDER':'BURGLRY'].sum(axis=1)/kills_state_crime.pop_avrg*100000

In [ ]:
violent_crimes.head(2)

In [ ]:
kills_state_crime['viol_crimes_100k'] = violent_crimes

In [ ]:
kills_state_crime['kills_per_1k'] = kills_state_crime.kills_total/kills_state_crime.pop_avrg*1000

In [ ]:
kills_state_crime.head()

In [ ]:
fig = px.choropleth(kills_state_crime, locations='state',
                    color='kills_per_1k',
                    hover_data=['STNAME', 'kills_per_1k', 'kills_total'],
                           color_continuous_scale="Purples",
                           scope="usa", locationmode="USA-states",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

fig = px.choropleth(kills_state_crime, locations='state',
                    color='viol_crimes_100k',
                    hover_data=['STNAME', 'kills_per_1k', 'viol_crimes_100k'],
                           color_continuous_scale="Purples",
                           scope="usa", locationmode="USA-states",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
kills_q = pd.qcut(kills_state_crime.kills_per_1k, q=4)
kills_state_crime['kills_quant'] = kills_q

In [ ]:
cq = pd.qcut(kills_state_crime.viol_crimes_100k.astype(int), q=4)
kills_state_crime['vc_quant'] = cq

In [ ]:
colormap = ['#e66101','#fdb863','#b2abd2','#5e3c99']

In [ ]:
kills_state_crime.head(2)

In [ ]:
kills_state_crime.info()

In [ ]:
fig = px.choropleth(kills_state_crime.sort_values(by='kills_quant', ascending=False), 
                    locations='state', color='kills_quant',
                    hover_data=['STNAME', 'pop_avrg', 'kills_per_1k', 'kills_total'],
                           color_discrete_sequence=colormap[::-1],
                           scope="usa", locationmode="USA-states",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

fig = px.choropleth(kills_state_crime.sort_values(by='vc_quant', ascending=False), 
                    locations='state', color='vc_quant',
                    hover_data=['STNAME', 'pop_avrg', 'kills_per_1k'],
                           color_discrete_sequence=colormap[::-1],
                           scope="usa", locationmode="USA-states",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Graphical distributions for different races

In [ ]:
# https://worldpopulationreview.com/states/states-by-race
race_pop_perc = pd.read_csv('../data/geodata/population_perc_per_state_per_race.csv')

In [ ]:
race_pop_perc.head(2)

In [ ]:
state_fips = pd.read_csv('../data/geodata/states_fips.csv', dtype=str)
state_fips.stusps = state_fips.stusps.str.strip()
state_fips.st = state_fips.st.str.strip()

In [ ]:
state_fips.head(1)

In [ ]:
race_pop_perc = race_pop_perc.merge(state_fips, how='left', left_on='State', right_on='stname')

In [ ]:
race_pop_perc = race_pop_perc.drop(labels=['stname'], axis=1).rename({'st': 'state_fips'}, axis=1)

In [ ]:
race_pop_perc.head(2)

In [ ]:
# Group main dataframe by race
kills_per_race = (df_fips.groupby(['race', 'state'])['name']
         .count()
         .reset_index()
         .rename({'name': 'kills_total'}, axis=1)
        )

In [ ]:
kills_per_race

In [ ]:
kills_per_race = kills_per_race.query('race in ["black", "white", "native american"]').reset_index().drop('index', axis=1)

In [ ]:
# merge with population avrg data

In [ ]:
pop_race = pd.merge(kills_state_pop, race_pop_perc, left_on='STNAME', right_on='State', how='inner')

In [ ]:
kills_state_pop.head(1)

In [ ]:
pop_race = pop_race.drop(labels=['STNAME', 'state_fips_y', 'stusps', 
                                 'AsianTotalPerc','HawaiianTotalPerc','OtherTotalPerc'], axis=1).rename({
            'state_fips_x': 'state_fips', 'state': 'state_usps'}, axis=1)

In [ ]:
pop_race.head(2)

In [ ]:
kills_per_race.head()

In [ ]:
kills_race_pop = kills_per_race.merge(pop_race, how='left', left_on='state', right_on='state_usps')

In [ ]:
kills_race = kills_race_pop.merge(crime_state, how='left', left_on='state_fips', right_on='FIPS_ST')

In [ ]:
kills_race.shape